In [5]:
import duckdb

In [6]:
db = duckdb.connect("../data/chess.duckdb")
db.sql("SET TimeZone = 'UTC'")

In [3]:
db.close()

In [7]:
db.sql("select * from information_schema.tables")

┌───────────────┬────────────────────┬─────────────────────┬───┬──────────┬───────────────┬───────────────┐
│ table_catalog │    table_schema    │     table_name      │ … │ is_typed │ commit_action │ TABLE_COMMENT │
│    varchar    │      varchar       │       varchar       │   │ varchar  │    varchar    │    varchar    │
├───────────────┼────────────────────┼─────────────────────┼───┼──────────┼───────────────┼───────────────┤
│ chess         │ chess_data_core    │ games               │ … │ NO       │ NULL          │ NULL          │
│ chess         │ chess_data_core    │ player_games        │ … │ NO       │ NULL          │ NULL          │
│ chess         │ chess_data_prep    │ game_moves          │ … │ NO       │ NULL          │ NULL          │
│ chess         │ chess_data_raw     │ players_games       │ … │ NO       │ NULL          │ NULL          │
│ chess         │ chess_data_raw     │ players_profiles    │ … │ NO       │ NULL          │ NULL          │
│ chess         │ chess_data

In [18]:
db.sql("""
select
uuid
from chess_data_core.games
where player_result <> 'checkmated'
and opponent_result <> 'checkmated'
and len(checkmate_pieces) > 0
""").to_df()

,uuid


In [19]:
db.sql("""
select
uuid
from chess_data_core.games
where (player_result = 'checkmated'
or opponent_result = 'checkmated')
and len(checkmate_pieces) = 0
""").to_df()

,uuid
